# OraculumLocalBox for _pwsh_

There are two way to setup and experiment with a local installation of Oraculum and Sibylla. This interactive notebook is designed to explain how to use PowerShell to install and manage Oraculum and Sibylla on a local machine.

You can run these commands interactively in a PowerShell terminal without need for this notebook. However, this notebook is designed to be a step-by-step guide to setting up a local installation of Oraculum and Sibylla.

You need to install _Docker_ on your system and WSL2 if you are on Windows. You can find instructions for installing Docker on your system [here](https://docs.docker.com/get-docker/). You can find instructions for installing WSL2 on Windows [here](https://docs.microsoft.com/en-us/windows/wsl/install-win10).

Depending on whether you are in *Windows* or *macOS*, the relative file reference changes appropriately, for example: ```./weaviate.secret.yml``` becomes ```.\weaviate.secret.yml```


## Keys

In this notebook we reference _oraculum.secret.conf_ and _weaviate.secret.yml_. Those are copies of _oraculum.conf_ and _weaviate.yml_ where you should indicate you OpenAI org id and key. You can get those from the [OpenAI dashboard](https://platform.openai.com/account/api-keys).

## First run

The initial deployment of _Oraculum_ requires the schema initialization first, then the Facts data structure can be populated. Follow the steps (in some cases a timeout may require re-evaluate the cell, or even restarting the kernels if stuck in evaluation):

In [1]:
# Start the _Weaviate_
docker-compose -f ./weaviate.secret.yml up -d

 Container weaviate  Running


You also need to install _Oraculum_ module from PowerShell Gallery. You can do that by running the following command in a PowerShell terminal:

In [2]:
Install-Module Oraculum


Untrusted repository
You are installing the modules from an untrusted repository. If you trust this repository, change its InstallationPolicy value by running the Set-PSRepository cmdlet. Are you sure you want to install the modules from 'PSGallery'?
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'N')

In [1]:
Connect-Oraculum -ConfigFile .\oraculum.secret.conf


Configuration          Logger                                               IsConnected
-------------          ------                                               -----------
Oraculum.Configuration Microsoft.Extensions.Logging.Abstractions.NullLogger        True



To initialize or reset the schema you use the `Reset-Schema` command. Notice that you will be prompted for entering an OTP to avoind unwanted exectution. Inside Visual Studio Code pay attention to the top of the window where an input box will be shown:

In [3]:
Reset-Schema


Schema reset
To reset the schema type in the code 984586
True


Now can we import Oraculum module and connect with the previously filled secrets

In [4]:
Import-Module Oraculum

Connect-Oraculum -ConfigFile ./oraculum.secret.conf


Configuration          IsConnected
-------------          -----------
Oraculum.Configuration        True



Now you can check that the fact class is empty:

In [2]:
Get-TotalFact

0


## Manage facts repository in _Oraculum_

Now that you have the system up and running you can start adding facts to the repository. You can do that by using the `Add-Fact` command. The command takes a single parameter, the fact to be added.

Remember to import the module and connect to _Oraculum_ first:

In [6]:
Import-Module Oraculum
Connect-Oraculum -ConfigFile ./oraculum.secret.conf


Configuration          IsConnected
-------------          -----------
Oraculum.Configuration        True




We first load a set of FAQs from a JSON file:

In [3]:
$json = Get-Content ./qamoviedb.json | ConvertFrom-Json
$n = 1
$json |% { Add-Fact -Category movie -FactType faq -Title $_.q -Content $_.a -Citation ("FAQ#" + $n++) }


Guid
----
93776f07-2ebe-474d-bb3f-2e41496e2c86
e44cc7a7-0ed2-41aa-96c8-014708768132
0df441c9-98b9-4737-a4fb-92ddb2aad9ab
4d03ba8e-0358-4303-bfde-c09d3dde9faf
b9f03dfc-62c3-4623-9be6-3564af2c4892
de2f1230-ffc1-46a5-a889-0b3138e4ba37
00ffaef9-cd0c-47dd-85f0-cd3c13c5c341
835fbae3-a6d2-4186-ba00-7894a4221d60
eb77b05e-7322-4494-b0d2-fd827d2d5b58
575e9710-1476-453f-b8c5-3ce712baad32
ef089850-d1e9-4069-874d-7760eb3d96f6
fe27575e-b750-44da-8e42-fd0a65270a1a
5a4f3322-964b-4249-a0c9-881abc3f1600
d051d87c-7918-428c-a362-d08f474d1f32
05bc504a-ebab-4854-bca7-73e14fcaeeef
5a62515d-8f16-4551-b955-06caf0f3bff7
21129ec7-b79e-457f-82ca-99e7fae01f6e
e456cae3-3efc-4c52-9a00-e57f7bb8e533
79093235-c2c3-463d-977b-2ab4c80a6199
3cc79ad2-d4ba-4d09-bc14-5dd78696cff6
2e9bc66d-0047-4580-bb46-24045fc5a3a3
e5dc82fc-0ed8-4150-9d3c-2e34b78c298a
e8ce43fb-8ed5-442b-a6fd-c83b285ac3c2
70cfa4c3-5359-478d-ac2d-b74ef098ec92
fb99c4aa-a472-4927-ae72-fc3afee26270
59e74b0e-26c4-45a1-b4d6-289fe07f0682
cdbd1852-0e94-48c8-ab6f-3a8

Error: Command failed: SubmitCode: $json = Get-Content ./qamoviedb.json | ConvertFrom ...

Note that we have added facts individually, this may take a while. The command `Add-Facts` would have been much faster but we should have created an array of `Fact` objects first. For small loads it is easier to add facts individually.

Now we can check that the facts have been added:

In [8]:
Get-TotalFact

287


You can use the `Get-Fact` command to retrieve a fact by its id, and the `Get-Facts` command to retrieve all facts in the repository. The `Get-Facts` command takes a single parameter, the number of facts to retrieve. If you do not specify a number, the limit is set to 1024:

In [9]:
Get-Facts -Limit 5


id               : 0029356a-055b-4105-83a2-51fb475f1408
distance         : 
factType         : faq
category         : movie
tags             : 
title            : The annual "awards" given out around the same time as the Oscars that “honor” 
                   the best of the worst in film, acting, and directing celebrated their 40th 
                   anniversary in 2020. What is the name of these "awards?"
content          : The Golden Raspberries (Razzies)
citation         : FAQ#97
reference        : 
expiration       : 
location         : 
locationDistance : 
locationName     : 
editPrincipals   : 
validFrom        : 
validTo          : 
factAdded        : 12/20/2023 1:24:41 PM

id               : 0087916e-febd-4624-a7ec-8abb921ff443
distance         : 
factType         : faq
category         : movie
tags             : 
title            : Though she gave it to Obi-Wan shortly before she died, Padmé was buried with a 
                   good luck charm that had been made by Anakin

If you are running in a windows terminal you can use the `Out-GridView` command to display the facts in a separate window:

    Get-Facts | Out-GridView

_Oraculum_ uses _Weaviate_ ability to order facts by the embedding of the query (closer vectors are more relevant *even if the query is expressed in a different language than the one of the facts*). The Find-RelevantFacts command is used to see the distance between a query and the facts in the repository:

In [10]:
Find-RelevantFacts -Query "What do you know about Sarlacc?" -Limit 5 | select distance, title, content


distance title
-------- -----                                                                                     
   0.144 What is the name of the carnivorous creature from Tatooine that C-3PO is talking about wh…
   0.199 In the affectionate Star Wars parody, “Space Balls,” the film renames the slimy crime lor…
   0.204 Jabba The Hut’s pet/sidekick in "Return Of The Jedi", a noisy little rodent, is named Sal…
   0.211 George Lucas is on record saying that its shape was inspired by a hamburger, how is the l…
   0.213 What hyperspace route passing by the Maw nebula did Han Solo traverse in the Millenium Fa…



## Starting a Sibylla session

You can easily test your Sibylla session by using the `New-SibyllaSession` command. This command takes a single parameter, the configuration of the session. The command will interactively ask for prompts and will display answers until `#quit` will not be entered (again pay attention to the top of the Visual Studio Code window for the input box):

In [14]:
New-SibyllaSession -ConfigFile ./sibylla.conf

Assistant: Hello from your personal movie assistant. I know curiosities about 287 movies, try me!
You: Assistant: Did you know that the 2015 Star Wars movie directed by J.J. Abrams, "The Force Awakens," was the first film in the franchise that didn't have a video game tie-in when it came out? However, it did eventually get a video game, but just the Lego version. [FAQ#204]
You: 